In [19]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import pyodbc

In [23]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=prdsql05.westfund.com.au;'
    'DATABASE=BRONZE;'
    'Trusted_Connection=yes;'
)

In [24]:
query = """
SELECT *
  FROM [GOLD].[dbo].[Membership_Reporting]
  WHERE [Run Month Num] = (SELECT MAX([Run Month Num]) FROM [GOLD].[dbo].[Membership_Reporting])
    AND [Product Cover] <> 'Overseas' AND [active_count] <> 0
"""

df_mem = pd.read_sql(query, conn)
len(df_mem)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_14144\720937309.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mem = pd.read_sql(query, conn)


67991

In [25]:
df_mem.head()

,Run Month,Run Month Num,Fin Year,Fin Month,Fin Month Order,MembershipKey,Membership Number,Membership Status,Product Cover,Membership Cover,...,Previous Fund,Join Reason,Termination Code,Termination Reason,Transfered Fund Id,Transfered Fund,active_count,join_count,suspended_count,terminated_count
0,2026-01-31,62,2026,Jan,7,81486PK 2026-01-31,81486.0,A,Combined,Hosp/Extras,...,Other,Transfer from another fund,None,None,NaN,None,1,0,0,0
1,2026-01-31,62,2026,Jan,7,84221PK 2026-01-31,84221.0,A,Combined,Hosp/Extras,...,Medibank,Transfer from another fund,None,None,NaN,None,1,0,0,0
2,2026-01-31,62,2026,Jan,7,84251PK 2026-01-31,84251.0,A,Combined,Hosp/Extras,...,Other,Transfer from another fund,None,None,NaN,None,1,0,0,0
3,2026-01-31,62,2026,Jan,7,81531PK 2026-01-31,81531.0,A,Combined,Hosp/Extras,...,Other,Transfer from another fund,None,None,NaN,None,1,0,0,0
4,2026-01-31,62,2026,Jan,7,84217PK 2026-01-31,84217.0,A,Combined,Hosp/Extras,...,Other,Transfer from another fund,None,None,NaN,None,1,0,0,0


In [26]:
query_2 = """
SELECT DISTINCT
[person_id],
[membership_id]

  FROM [BRONZE].[dbo].[person_membership]

  WHERE [membership_id] IN (SELECT [Membership Number]
  FROM [GOLD].[dbo].[Membership_Reporting]
  WHERE [Run Month Num] = (SELECT MAX([Run Month Num]) FROM [GOLD].[dbo].[Membership_Reporting])
    AND [Product Cover] <> 'Overseas' AND [active_count] <> 0)
"""

df_per_mem = pd.read_sql(query_2, conn)
len(df_per_mem)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_14144\2215566542.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_per_mem = pd.read_sql(query_2, conn)


175020

In [27]:
df_per_mem

,person_id,membership_id
0,35529.0,55652.0
1,189122.0,109536.0
2,33955.0,55017.0
3,81329.0,158341.0
4,350406.0,201238.0
...,...,...
175015,55367.0,63013.0
175016,30124.0,230859.0
175017,83892.0,132158.0
175018,414871.0,200249.0


In [28]:
query = """
SELECT *
  FROM [BRONZE].[dbo].[PersonAddressHomePostal]
"""

df = pd.read_sql(query, conn)
len(df)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_14144\2897101254.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


172712

In [29]:
keys = df[['person_id', 'membership_id']].drop_duplicates()

mask = df_per_mem[['person_id', 'membership_id']].apply(tuple, axis=1).isin(
    keys.apply(tuple, axis=1)
)

df_matched = df_per_mem[mask]
df_unmatched = df_per_mem[~mask]

num_unmatched = (~mask).sum()



In [30]:
len(df_matched)

70468

##### can use [BRONZE].[dbo].[person_membership], [BRONZE].[dbo].[PersonContact],[BRONZE].[dbo].[PersonAddressHomePostal]

In [31]:
df_unmatched
df_unmatched[df_unmatched["membership_id"] == 61319]


,person_id,membership_id
1232,51407.0,61319.0
55634,51406.0,61319.0


In [32]:
df_sql_sa = pd.read_csv(r'\\prdeqs01\QlikData\GEO\output\df_sql_with_sa_03022026.csv')
df_sql_sa

,person_id,membership_id,state,suburb,postcode,address_line,long,lat,SA1_CODE,SA2_CODE,SA3_CODE,SA4_CODE,SA2_NAME,SA3_NAME,SA4_NAME
0,238754.0,145870.0,QLD,MAROOCHYDORE,4558,19 Schirrmann Dr,153.080047,-26.648962,3.160314e+10,316031427.0,31603.0,316.0,Maroochydore - Kuluin,Maroochy,Sunshine Coast
1,238809.0,145901.0,NSW,PORT KEMBLA,2505,23 Fifth Ave,150.903987,-34.485314,1.070115e+10,107011546.0,10701.0,107.0,Port Kembla - Warrawong,Dapto - Port Kembla,Illawarra
2,238840.0,145915.0,QLD,JUBILEE POCKET,4802,40 Moonlight Drive,148.720346,-20.284680,3.120314e+10,312031359.0,31203.0,312.0,Airlie - Whitsundays,Whitsunday,Mackay - Isaac - Whitsunday
3,237679.0,145327.0,QLD,EAST MACKAY,4740,6 Hoey St,149.198695,-21.154226,3.120213e+10,312021343.0,31202.0,312.0,East Mackay,Mackay,Mackay - Isaac - Whitsunday
4,237815.0,145388.0,QLD,KOUMALA,4738,1220 Turnors Paddock Road,149.233704,-21.659315,3.120214e+10,312021352.0,31202.0,312.0,Sarina,Mackay,Mackay - Isaac - Whitsunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70463,238478.0,145726.0,QLD,MORANBAH,4744,3 Leichhardt Drive,148.041075,-21.995782,3.120113e+10,312011341.0,31201.0,312.0,Moranbah,Bowen Basin - North,Mackay - Isaac - Whitsunday
70464,238569.0,145787.0,NSW,ELANORA HEIGHTS,2101,4 Dewrang Ave,151.281248,-33.704838,1.220214e+10,122021421.0,12202.0,122.0,Bayview - Elanora Heights,Pittwater,Sydney - Northern Beaches
70465,238600.0,145802.0,NSW,BROKEN HILL,2880,153 Cornish Street,141.437527,-31.970293,1.050211e+10,105021097.0,10502.0,105.0,Broken Hill,Broken Hill and Far West,Far West and Orana
70466,238658.0,145833.0,NSW,CREMORNE POINT,2090,Unit 5 47-55 Milson Rd,151.227158,-33.843108,1.210414e+10,121041413.0,12104.0,121.0,Cremorne - Cammeray,North Sydney - Mosman,Sydney - North Sydney and Hornsby


In [33]:
query_person_membership = """
SELECT [person_id]
      ,[membership_id]
      ,[person_memship_version]
      ,[relationship]
      ,[status_flag]
FROM [BRONZE].[dbo].[person_membership]
"""

df_person_membership = pd.read_sql(query_person_membership, conn)
df_person_membership

C:\Users\zengsh\AppData\Local\Temp\ipykernel_14144\1723840129.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_person_membership = pd.read_sql(query_person_membership, conn)


,person_id,membership_id,person_memship_version,relationship,status_flag
0,251365.0,1.0,15.0,1,A
1,270473.0,1.0,13.0,2,A
2,353483.0,1.0,3.0,4,T
3,353524.0,1.0,3.0,4,T
4,378793.0,1.0,8.0,5,T
...,...,...,...,...,...
394126,447604.0,240407.0,3.0,1,A
394127,447605.0,240408.0,3.0,1,A
394128,447615.0,240409.0,3.0,1,A
394129,447616.0,240410.0,3.0,1,A


In [34]:
# 检查 [person_id, membership_id, relationship] 组合是否有重复
duplicates = df_person_membership.duplicated(subset=['person_id', 'membership_id', 'relationship'], keep=False)
num_duplicates = duplicates.sum()

print(f"总行数: {len(df_person_membership)}")
print(f"重复的行数: {num_duplicates}")
print(f"唯一组合数: {df_person_membership[['person_id', 'membership_id', 'relationship']].drop_duplicates().shape[0]}")

# 如果有重复，显示重复的记录
if num_duplicates > 0:
    print("\n重复的记录:")
    df_person_membership[duplicates].sort_values(['person_id', 'membership_id', 'relationship'])

总行数: 394131
重复的行数: 0
唯一组合数: 394131


In [35]:
# 通过 person_id 和 membership_id 合并 relationship 列
df_sql_sa_with_rel = df_sql_sa.merge(
    df_person_membership[['person_id', 'membership_id', 'relationship']],
    on=['person_id', 'membership_id'],
    how='left'
)

print(f"合并前行数: {len(df_sql_sa)}")
print(f"合并后行数: {len(df_sql_sa_with_rel)}")
print(f"relationship 非空数量: {df_sql_sa_with_rel['relationship'].notna().sum()}")
df_sql_sa_with_rel.head()

合并前行数: 70468
合并后行数: 70468
relationship 非空数量: 70468


,person_id,membership_id,state,suburb,postcode,address_line,long,lat,SA1_CODE,SA2_CODE,SA3_CODE,SA4_CODE,SA2_NAME,SA3_NAME,SA4_NAME,relationship
0,238754.0,145870.0,QLD,MAROOCHYDORE,4558,19 Schirrmann Dr,153.080047,-26.648962,3.160314e+10,316031427.0,31603.0,316.0,Maroochydore - Kuluin,Maroochy,Sunshine Coast,1
1,238809.0,145901.0,NSW,PORT KEMBLA,2505,23 Fifth Ave,150.903987,-34.485314,1.070115e+10,107011546.0,10701.0,107.0,Port Kembla - Warrawong,Dapto - Port Kembla,Illawarra,1
2,238840.0,145915.0,QLD,JUBILEE POCKET,4802,40 Moonlight Drive,148.720346,-20.284680,3.120314e+10,312031359.0,31203.0,312.0,Airlie - Whitsundays,Whitsunday,Mackay - Isaac - Whitsunday,1
3,237679.0,145327.0,QLD,EAST MACKAY,4740,6 Hoey St,149.198695,-21.154226,3.120213e+10,312021343.0,31202.0,312.0,East Mackay,Mackay,Mackay - Isaac - Whitsunday,1
4,237815.0,145388.0,QLD,KOUMALA,4738,1220 Turnors Paddock Road,149.233704,-21.659315,3.120214e+10,312021352.0,31202.0,312.0,Sarina,Mackay,Mackay - Isaac - Whitsunday,1


In [36]:
# 检查 membership_id 重复情况
total_rows = len(df_sql_sa_with_rel)
unique_membership_ids = df_sql_sa_with_rel['membership_id'].nunique()
duplicated_count = df_sql_sa_with_rel['membership_id'].duplicated().sum()

print(f"总行数: {total_rows}")
print(f"唯一 membership_id 数量: {unique_membership_ids}")
print(f"重复的 membership_id 行数: {duplicated_count}")
print(f"重复率: {duplicated_count / total_rows * 100:.2f}%")

# 显示重复的 membership_id 及其出现次数
dup_counts = df_sql_sa_with_rel['membership_id'].value_counts()
dup_counts[dup_counts > 1]

总行数: 70468
唯一 membership_id 数量: 67991
重复的 membership_id 行数: 2477
重复率: 3.52%


membership_id
222178.0    6
102213.0    5
126986.0    4
188979.0    4
172907.0    4
           ..
112271.0    2
161582.0    2
162297.0    2
161905.0    2
162345.0    2
Name: count, Length: 2393, dtype: int64

In [37]:
# 按 relationship 排序后，保留每个 membership_id 中 relationship 最小的行
df_sql_sa_dedup = (
    df_sql_sa_with_rel
    .sort_values('relationship')
    .drop_duplicates(subset='membership_id', keep='first')
    .reset_index(drop=True)
)

print(f"去重前行数: {len(df_sql_sa_with_rel)}")
print(f"去重后行数: {len(df_sql_sa_dedup)}")
print(f"移除了 {len(df_sql_sa_with_rel) - len(df_sql_sa_dedup)} 行重复数据")
df_sql_sa_dedup.head()

去重前行数: 70468
去重后行数: 67991
移除了 2477 行重复数据


,person_id,membership_id,state,suburb,postcode,address_line,long,lat,SA1_CODE,SA2_CODE,SA3_CODE,SA4_CODE,SA2_NAME,SA3_NAME,SA4_NAME,relationship
0,238754.0,145870.0,QLD,MAROOCHYDORE,4558,19 Schirrmann Dr,153.080047,-26.648962,3.160314e+10,316031427.0,31603.0,316.0,Maroochydore - Kuluin,Maroochy,Sunshine Coast,1
1,436328.0,236061.0,QLD,VALENTINE PLAINS,4715,252 Baileys Lane,150.556627,-24.416995,3.080415e+10,308041528.0,30804.0,308.0,Banana,Biloela,Central Queensland,1
2,436336.0,236068.0,NSW,BRANXTON,2335,4 Highfield Way,151.363677,-32.638614,1.060111e+10,106011107.0,10601.0,106.0,Branxton - Greta - Pokolbin,Lower Hunter,Hunter Valley exc Newcastle,1
3,436444.0,236091.0,NSW,WEST HOXTON,2171,19 Manton Avenue,150.833271,-33.934445,1.270116e+10,127011597.0,12701.0,127.0,West Hoxton - Middleton Grange,Bringelly - Green Valley,Sydney - South West,1
4,436531.0,236124.0,QLD,GARBUTT,4814,41 Sunderland Street,146.775161,-19.255600,3.180215e+10,318021475.0,31802.0,318.0,Garbutt - West End,Townsville,Townsville,1


In [38]:
# 验证去重后 membership_id 是否还有重复
duplicated_count = df_sql_sa_dedup['membership_id'].duplicated().sum()
print(f"去重后 membership_id 重复行数: {duplicated_count}")
print(f"总行数: {len(df_sql_sa_dedup)}")
print(f"唯一 membership_id 数量: {df_sql_sa_dedup['membership_id'].nunique()}")

去重后 membership_id 重复行数: 0
总行数: 67991
唯一 membership_id 数量: 67991


In [39]:
# 导出 CSV，使用动态日期后缀
date_suffix = datetime.now().strftime('%d%m%Y')
output_path = rf'\\prdeqs01\QlikData\GEO\output\df_sql_with_sa_unique_{date_suffix}.csv'

df_sql_sa_dedup.to_csv(output_path, index=False)
print(f"已导出到: {output_path}")
print(f"导出行数: {len(df_sql_sa_dedup)}")

已导出到: \\prdeqs01\QlikData\GEO\output\df_sql_with_sa_unique_03022026.csv
导出行数: 67991
